## Adding a timestamp when you index documents

This notebook shows how you can add custom fields and actions to run automatically when you ingest data with the Python client. 

You can store a timestamp each time a document in your index is modified.

Using the concepts of: [the ingest pipeline](https://www.elastic.co/guide/en/elasticsearch/reference/8.12/ingest.html) and [the set processor.](https://www.elastic.co/guide/en/elasticsearch/reference/current/set-processor.html)

In [2]:
from getpass import getpass  
from elasticsearch import Elasticsearch

# Prompt the user to enter their Elastic Cloud ID and API Key securely
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

We want to create two fields for our index to keep track of when it was created (a fixes date), and when it was last updated (new date with every change)

In [85]:
index_name = "test_timestamp"

mappings = {
    "properties" : {
        "foo" : {
            "type" : "keyword",
            "type" : "text"
        },
        "created_at": {
            "type": "date" 
        },
        "updated_at" : {
            "type" : "date"
        }
    }
}

settings = {
    "index.default_pipeline" : "ingest_with_dates"
}

client.ingest.put_pipeline(
    id="ingest_with_dates", 
    processors=[
    {
        "set": {
            "field": "updated_at",
            "value": "{{_ingest.timestamp}}"
        }
    },{
        "set": {
            "if" : "ctx?.created_at == null",
            "field": "created_at",
            "value": "{{_ingest.timestamp}}"
        }
    }]
)


client.indices.create(index=index_name, mappings=mappings, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_timestamp'})

In [90]:
client.index(
    index=index_name,
    id=0,
    document={
        "foo": "bar",
    },
)

ObjectApiResponse({'_index': 'test_timestamp', '_id': '0', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [32]:
query={
    "match": {
        "foo": "barr"
    }
}

response = client.search(index=index_name, query=query)
for hit in response["hits"]["hits"]:
    print(hit['_source'])

{'updated_at': '2024-03-12T17:14:44.962992392Z', 'created_at': '2024-03-12T17:14:44.962992392Z', 'foo': 'barr'}


In [94]:
client.update(index=index_name, id = 1, body={"doc":{"foo" : "bar+"}})

ObjectApiResponse({'_index': 'test_timestamp', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1})

In [95]:
query={
    "match": {
        "_id": 0
    }
}

response = client.search(index=index_name, query=query)
for hit in response["hits"]["hits"]:
    print(hit['_source'])

{'updated_at': '2024-03-12T15:52:07.044704541Z', 'created_at': '2024-03-12T15:51:39.457194217Z', 'foo': 'bar+'}
